In [3]:
def compare_date_components(day_doc, month_doc, year_doc, day_req, month_req, year_req):
    """Compare two date components.

    Args:
        day_doc (int): Day of the document.
        month_doc (int): Month of the document.
        year_doc (int): Year of the document.
        day_req (int): Day of the request.
        month_req (int): Month of the request.
        year_req (int): Year of the request.

    Returns:
        int: -1 if date_doc < date_requete, 0 if equal, 1 if date_doc > date_requete.
    """
    if year_doc < year_req:
        return -1
    elif year_doc > year_req:
        return 1
    else:
        if month_doc < month_req:
            return -1
        elif month_doc > month_req:
            return 1
        else:
            if day_doc < day_req:
                return -1
            elif day_doc > day_req:
                return 1
            else:
                return 0
def compare_dates(date_doc, date_requete):
    """ date_requete = resultat = {"début": None, "fin": None, "précis": None, "not": None}
    avec value de chaque key = {"j": None, "m": None, "a": None} 
    date_doc = 'j/m/a'
    Args:
        date1 (str): The date of the doucument to compare; format 'j/m/a'.
        date2 (str): The date de la requête à comparer; format {"début": None, "fin": None, "précis": None, "not": None}
            with each value being a dictionary: {"j": None, "m": None, "a": None}.

    Returns:
        True if date_doc fits date_requete, False otherwise.
    """
    # Split the dates into components
    day_doc, month_doc, year_doc = date_doc.split('/')

    #check date_requete empty aka no restriction aka always True
    if date_requete["début"] is None and date_requete["fin"] is None and date_requete["précis"] is None and date_requete["not"] is None:
        return True

    #check date_requete["not"]
    if date_requete["not"] is not None: 
        day_not, month_not, year_not = int(date_requete["not"]["j"]), int(date_requete["not"]["m"]), int(date_requete["not"]["a"])
        if day_doc == day_not and month_doc == month_not and year_doc == year_not:
            return False

    #compare date_requete["précis"]
    if date_requete["précis"] is not None:
        day_req, month_req, year_req = int(date_requete["précis"]["j"]), int(date_requete["précis"]["m"]), int(date_requete["précis"]["a"])
        if day_doc == day_req and month_doc == month_req and year_doc == year_req:
            return True
        else:
            return False
        
    res = True
    # Compare date_requete["début"]
    if date_requete["début"] is not None:
        day_req, month_req, year_req = int(date_requete["début"]["j"]), int(date_requete["début"]["m"]), int(date_requete["début"]["a"])
        if compare_date_components(day_doc, month_doc, year_doc, day_req, month_req, year_req) < 0:
            res = res and False
    
    # Compare date_requete["fin"]
    if date_requete["fin"] is not None:
        day_req, month_req, year_req = int(date_requete["fin"]["j"]), int(date_requete["fin"]["m"]), int(date_requete["fin"]["a"])
        if compare_date_components(day_doc, month_doc, year_doc, day_req, month_req, year_req) > 0:
            res = res and False
    
    # Dates are equal
    return res

In [ ]:
import pandas as pd
#rubrique et dates dans others.csv
# mots_clés dans text_title.csv

'''
    resultats = {
        "return": None,
        "mots_cles": {"yes": [], "no": None},
        "operateurs_mots_cles": None,
        "rubrique": None,
        "operateurs_rubrique"
        "dates": {"debut": None, "fin": None, "précis": None, "not": None},
        "titre": None,
        "operateurs_titre": None,
        "images": None,
    }

'''
def recherche_documents(resultats, index_inverse_texte, index_inverse_date, index_inverse_rubrique, index_inverse_titre):
    """
    Fonction qui recherche les documents correspondant, dans lemmes_path, aux résultats de la requete (resultats).
    :param resultats: dictonnaires contenant les résultats de la requête
    :param index_inverse_mots_cles: fichier csv index inversé des mots-clés
    :param index_inverse_dates_rubriques: fichier csv index inversé des dates et rubriques
    :return: Liste de documents correspondant aux résultats de la requête
    """
    docs_cherches = []
    #partie mots-clés (sans les not)
    mots_cles = resultats["mots_cles"]
    if (len(mots_cles["yes"]) != 0) or (mots_cles["no"] != None):
        pass
    else:
        dict_match_key = {}
        index_texte = pd.read_csv(index_inverse_texte, sep= "\t")
        for mot in mots_cles["yes"]:
            docs_qui_matchs = index_texte.loc[index_texte["mot"] == mot, "docs"].values[0]
            dict_match_key[mot] = docs_qui_matchs
        if resultats["operateurs_mots_cles"] == "ou":
            docs_key = list(set().union(set(l) for l in dict_match_key.items()))
        else:
            docs_key = list(set.intersection((set(l) for l in dict_match_key.items())))

        if len(docs_cherches) == 0:
            docs_cherches = docs_key
        else:
            docs_cherches = list(set(docs_cherches) & set(docs_key))

    #partie titre
    titres = resultats["titre"]
    if titres != None:
        dict_match_titre = {}
        index_titre = pd.read_csv(index_inverse_titre, sep='\t')
        if isinstance(titres) != list:
            titres = [].append(titres)
        for titre in titres:
            docs_qui_matchs = index_titre.loc[index_titre["mot"] == titre, "docs"].values[0]
            dict_match_titre[titre] = docs_qui_matchs
        if resultats["operateurs_titre"] == "ou":
            docs_titre = list(set().union(set(l) for l in dict_match_titre.items()))
        else:
            docs_titre = list(set.intersection((set(l) for l in dict_match_titre.items())))
        
        if len(docs_cherches) == 0:
            docs_cherches = docs_titre
        else:
            docs_cherches = list(set(docs_cherches) & set(docs_titre))
    
    #partie rubrique
    rubriques = resultats["rubrique"]
    if rubriques != None:
        dict_match_rubrique = {}
        index_rubrique = pd.read_csv(index_inverse_rubrique, sep='\t')
        if isinstance(rubriques) != list:
            rubriques = [].append(rubriques)
        for rubrique in rubriques:
            docs_qui_matchs = index_rubrique.loc[index_rubrique["mot"] == rubrique, "docs"].values[0]
            dict_match_rubrique[rubrique] = docs_qui_matchs
        if resultats["operateurs_titre"] == "ou":
            docs_titre = list(set().union(set(l) for l in dict_match_rubrique.items()))
        else:
            docs_titre = list(set.intersection((set(l) for l in dict_match_rubrique.items())))
        
        if len(docs_cherches) == 0:
            docs_cherches = docs_titre
        else:
            docs_cherches = list(set(docs_cherches) & set(docs_titre))

    #partie images
    image = resultats["images"]
    if image != None:
        


    return docs_cherches

In [ ]:
from TD6.td6 import *
from TD5.td5 import *

if __name__ == "__main__":
    # extraction des informations de la requete
    requete = input("Entrez votre requête en langage naturel : ")
    resultat = traiter_requete(requete)

    print("Représentation structurée de la requête :")
    print(resultat)
    # correction orthographique des mots clés
    lemmes_path = "TD4\lemmes.csv"
    if resultat["mots_cles"]["no"] != None:
        resultat["mots_cles"]["no"] = correction_orthographique(resultat["mots_cles"]["no"], lemmes_path)
    
    for i in range(len(resultat["mots_cles"]["yes"])):
        resultat["mots_cles"]["yes"][i] = correction_orthographique(resultat["mots_cles"]["yes"][i], lemmes_path)
    
    print("Requête corrigée :")
    print(resultat)

    # recherche des documents pertinents
    documents = recherche_documents(resultat, lemmes_path)
